# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

In [ ]:
#Import your libraries

import numpy as np
import pandas as pd

# Introduction

In this lab, we will use two datasets. Both datasets contain variables that describe apps from the Google Play Store. We will use our knowledge in feature extraction to process these datasets and prepare them for the use of a ML algorithm.

# Challenge 1 - Loading and Extracting Features from the First Dataset

#### In this challenge, our goals are: 

* Exploring the dataset.
* Identify the columns with missing values.
* Either replacing the missing values in each column or drop the columns.
* Conver each column to the appropriate type.

#### The first dataset contains different information describing the apps. 

Load the dataset into the variable `google_play` in the cell below. The dataset is in the file `googleplaystore.csv`

In [ ]:
# Your code here:

google_play = pd.read_csv('https://raw.githubusercontent.com/ta-data-par/DAFT_NOV_21_01/main/module_3/5.%20Feature-Extraction/data/googleplaystore.csv')

In [ ]:
google_play.head()

App  ...   Android Ver
0     Photo Editor & Candy Camera & Grid & ScrapBook  ...  4.0.3 and up
1                                Coloring book moana  ...  4.0.3 and up
2  U Launcher Lite – FREE Live Cool Themes, Hide ...  ...  4.0.3 and up
3                              Sketch - Draw & Paint  ...    4.2 and up
4              Pixel Draw - Number Art Coloring Book  ...    4.4 and up

[5 rows x 13 columns]

#### Examine all variables and their types in the following cell

In [ ]:
# Your code here:
google_play.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver',
       'Android Ver'],
      dtype='object')

In [ ]:
google_play.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

In [ ]:
google_play.App.value_counts()

ROBLOX                                               9
CBS Sports App - Scores, News, Stats & Watch Live    8
8 Ball Pool                                          7
Duolingo: Learn Languages Free                       7
Candy Crush Saga                                     7
                                                    ..
The Sims™ Mobile                                     1
TEKKEN™                                              1
HDWallpaper DK                                       1
App vault                                            1
Fantasy theme dark bw black building                 1
Name: App, Length: 9660, dtype: int64

#### Since this dataset only contains one numeric column, let's skip the `describe()` function and look at the first 5 rows using the `head()` function

In [ ]:
# Your code here:
google_play.head(2)

App  ...   Android Ver
0  Photo Editor & Candy Camera & Grid & ScrapBook  ...  4.0.3 and up
1                             Coloring book moana  ...  4.0.3 and up

[2 rows x 13 columns]

#### We can see that there are a few columns that could be coerced to numeric.

Start with the reviews column. We can evaluate what value is causing this column to be of object type finding the non-numeric values in this column. To do this, we recall the `to_numeric()` function. With this function, we are able to coerce all non-numeric data to null. We can then use the `isnull()` function to subset our dataframe using the True/False column that this function generates.

In the cell below, transform the Reviews column to numeric and assign this new column to the variable `Reviews_numeric`. Make sure to coerce the errors.

In [ ]:
# Your code here:
google_play['Reviews_numeric'] = pd.to_numeric(google_play['Reviews'], errors = 'coerce')

In [ ]:
Reviews_numeric.dtypes

NameError: ignored

Next, create a column containing True/False values using the `isnull()` function. Assign this column to the `Reviews_isnull` variable.

In [ ]:
# Your code here:
google_play['Reviews_isnull'] = pd.isna(google_play['Reviews_numeric'])
google_play.head()

Finally, subset the `google_play` with `Reviews_isnull`. This should give you all the rows that contain non-numeric characters.

Your output should look like:

![Reviews_bool.png](reviews-bool.png)

In [ ]:
# Your code here:
google_play[google_play['Reviews_isnull']==True]

#### We see that Google Play is using a shorthand for millions. 

Let's write a function to transform this data.

Steps:

1. Create a function that returns the correct numeric values of *Reviews*.
1. Define a test string with `M` in the last character.
1. Test your function with the test string. Make sure your function works correctly. If not, modify your functions and test again.

In [ ]:
# Your code here

def convert_string_to_numeric(s): 
    """
    Convert a string value to numeric. If the last character of the string is `M`, obtain the 
    numeric part of the string, multiply it with 1,000,000, then return the result. Otherwise, 
    convert the string to numeric value and return the result.
    
    Args:
        s: The Reviews score in string format.

    Returns:
        The correct numeric value of the Reviews score.
    """
    if s[-1]=='M':
      s=s[:-1]
      s=float(s)*1000000
    else:
      s=float(s)
    return s

    #return np.NaN

test_string = '4.0M'

convert_string_to_numeric(test_string) == 4000000

The last step is to apply the function to the `Reviews` column in the following cell:

In [ ]:
# Your code here:
google_play['Reviews']=list(map(convert_string_to_numeric,google_play['Reviews']))

Check the non-numeric `Reviews` row again. It should have been fixed now and you should see:

![Reviews_bool_fixed.png](reviews-bool-fixed.png)

In [ ]:
# Your code here
google_play[google_play['Reviews_isnull']==True]

Also check the variable types of `google_play`. The `Reviews` column should be a `float64` type now.

In [ ]:
# Your code here:
google_play.dtypes

#### The next column we will look at is `Size`. We start by looking at all unique values in `Size`:

*Hint: use `unique()` ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.unique.html))*.

In [ ]:
# Your code here:
google_play.Size.unique()

You should have seen lots of unique values of the app sizes.

#### While we can convert most of the `Size` values to numeric in the same way we converted the `Reviews` values, there is one value that is impossible to convert.

What is that badass value? Enter it in the next cell and calculate the proportion of its occurence to the total number of records of `google_play`.

In [ ]:
# Your code here:
# Your code here:
varies_count = google_play[google_play['Size']=='Varies with device']['Size'].count()
proportion = varies_count/google_play.shape[0]
print(proportion)

#### While this column may be useful for other types of analysis, we opt to drop it from our dataset. 

There are two reasons. First, the majority of the data are ordinal but a sizeable proportion are missing because we cannot convert them to numerical values. Ordinal data are both numerical and categorical, and they usually can be ranked (e.g. 82k is smaller than 91M). In contrast, non-ordinal categorical data such as blood type and eye color cannot be ranked. The second reason is as a categorical column, it has too many unique values to produce meaningful insights. Therefore, in our case the simplest strategy would be to drop the column.

Drop the column in the cell below (use `inplace=True`)

In [ ]:
# Your code here:
google_play.drop(['Size'], axis=1, inplace=True)
google_play.head()

#### Now let's look at how many missing values are in each column. 

This will give us an idea of whether we should come up with a missing data strategy or give up on the column all together. In the next column, find the number of missing values in each column: 

*Hint: use the `isna()` and `sum()` functions.*

In [ ]:
# Your code here:
google_play.isna().sum()

You should find the column with the most missing values is now `Rating`.

#### What is the proportion of the missing values in `Rating` to the total number of records?

Enter your answer in the cell below.

In [ ]:
# Your code here:
google_play['Rating'].isna().sum()/google_play.shape[0]*100

A sizeable proportion of the `Rating` column is missing. A few other columns also contain several missing values.

#### We opt to preserve these columns and remove the rows containing missing data.

In particular, we don't want to drop the `Rating` column because:

* It is one of the most important columns in our dataset. 

* Since the dataset is not a time series, the loss of these rows will not have a negative impact on our ability to analyze the data. It will, however, cause us to lose some meaningful observations. But the loss is limited compared to the gain we receive by preserving these columns.

In the cell below, remove all rows containing at least one missing value. Use the `dropna()` function ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html)). Assign the new dataframe to the variable `google_missing_removed`.

In [ ]:
# Your code here:
google_missing_removed=google_play.dropna()
print(google_missing_removed.shape)
print(google_missing_removed.isna().sum())

From now on, we use the `google_missing_removed` variable instead of `google_play`.

#### Next, we look at the `Last Updated` column.

The `Last Updated` column seems to contain a date, though it is classified as an object type. Let's convert this column using the `pd.to_datetime` function ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html)).

In [ ]:
# Your code here:
google_missing_removed['Last Updated']=pd.to_datetime(google_missing_removed['Last Updated'],format='%B %d, %Y')

#### The last column we will transform is `Price`. 

We start by looking at the unique values of this column.

In [ ]:
# Your code here:
google_missing_removed['Price'].unique()

NameError: ignored

Since all prices are ordinal data without exceptions, we can tranform this column by removing the dollar sign and converting to numeric. We can create a new column called `Price Numerical` and drop the original column.

We will achieve our goal in three steps. Follow the instructions of each step below.

#### First we remove the dollar sign. Do this in the next cell by applying the `str.replace` function to the column to replace `$` with an empty string (`''`).

In [ ]:
# Your code here:
google_missing_removed['Price Numerical']=google_missing_removed['Price'].str.replace('$','')

NameError: ignored

#### Second step, coerce the `Price Numerical` column to numeric.

In [ ]:
# Your code here:
google_missing_removed['Price Numerical'] = pd.to_numeric(google_missing_removed['Price Numerical'], errors = 'coerce')

NameError: ignored

**Finally, drop the original `Price` column.**

In [ ]:
# Your code here:
google_missing_removed.drop('Price', axis=1)
google_missing_removed

NameError: ignored

Now check the variable types of `google_missing_removed`. Make sure:

* `Size` and `Price` columns have been removed.
* `Rating`, `Reviews`, and `Price Numerical` have the type of `float64`.
* `Last Updated` has the type of `datetime64`.

In [ ]:
# Your code here
google_missing_removed.dtypes

NameError: ignored

# Challenge 2 - Loading and Extracting Features from the Second Dataset

Load the second dataset to the variable `google_reviews`. The data is in the file `googleplaystore_user_reviews.csv`.

In [ ]:
# Your code here:

google_review = pd.read_csv('https://raw.githubusercontent.com/ta-data-par/DAFT_NOV_21_01/main/module_3/5.%20Feature-Extraction/data/googleplaystore_user_reviews.csv')

#### This dataset contains the top 100 reviews for each app. 

Let's examine this dataset using the `head` function

In [ ]:
# Your code here
google_review.head()

App  ... Sentiment_Subjectivity
0  10 Best Foods for You  ...               0.533333
1  10 Best Foods for You  ...               0.288462
2  10 Best Foods for You  ...                    NaN
3  10 Best Foods for You  ...               0.875000
4  10 Best Foods for You  ...               0.300000

[5 rows x 5 columns]

#### The main piece of information we would like to extract from this dataset is the proportion of positive reviews of each app. 

Columns like `Sentiment_Polarity` and `Sentiment_Subjectivity` are not to our interests because we have no clue how to use them. We do not care about `Translated_Review` because natural language processing is too complex for us at present (in fact the `Sentiment`, `Sentiment_Polarity`, and `Sentiment_Subjectivity` columns are derived from `Translated_Review` the data scientists). 

What we care about in this challenge is `Sentiment`. To be more precise, we care about **what is the proportion of *Positive* sentiment of each app**. This will require us to aggregate the `Sentiment` data by `App` in order to calculate the proportions.

Now that you are clear about what we are trying to achieve, follow the steps below that will walk you through towards our goal.

#### Our first step will be to remove all rows with missing sentiment. 

In the next cell, drop all rows with missing data using the `dropna()` function and assign this new dataframe to `review_missing_removed`.

In [ ]:
# Your code here:
review_missing_removed=google_review.dropna()

#### Now, use the `value_counts()` function ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.value_counts.html)) to get a sense on how many apps are in this dataset and their review counts.

In [ ]:
# Your code here:
review_missing_removed.App.value_counts()

Bowmasters                                            312
Helix Jump                                            273
Angry Birds Classic                                   273
Calorie Counter - MyFitnessPal                        254
Duolingo: Learn Languages Free                        240
                                                     ... 
Best Fiends - Free Puzzle Game                          1
Fruit Block - Puzzle Legend                             1
Best Ovulation Tracker Fertility Calendar App Glow      1
Google Trips - Travel Planner                           1
Calendar+ Schedule Planner App                          1
Name: App, Length: 865, dtype: int64

#### Now the tough part comes. Let's plan how we will achieve our goal:

1. We will count the number of reviews that contain *Positive* in the `Sentiment` column.

1. We will create a new dataframe to contain the `App` name, the number of positive reviews, and the total number of reviews of each app.

1. We will then loop the new dataframe to calculate the postivie review portion of each app.

#### Step 1: Count the number of positive reviews.

In the following cell, write a function that takes a column and returns the number of times *Positive* appears in the column. 

*Hint: One option is to use the `np.where()` function ([documentation](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.where.html)).*

In [ ]:
review_missing_removed.Sentiment.value_counts()

Positive    23998
Negative     8271
Neutral      5158
Name: Sentiment, dtype: int64

In [ ]:
review_missing_removed['Positive']=0
review_missing_removed.loc[review_missing_removed["Sentiment"]=='Positive', 'Positive'] = 1
review_missing_removed.head(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


App  ... Positive
0  10 Best Foods for You  ...        1
1  10 Best Foods for You  ...        1

[2 rows x 6 columns]

In [ ]:
# Your code below

#def positive_function(x):
    """
    Count how many times the string `Positive` appears in a column (exact string match).
    
    Args:
        x: data column
    
    Returns:
        The number of occurrences of `Positive` in the column data.
    """
    #return 0

IndentationError: ignored

#### Step 2: Create a new dataframe to contain the `App` name, the number of positive reviews, and the total number of reviews of each app

We will group `review_missing_removed` by the `App` column, then aggregate the grouped dataframe on the number of positive reviews and the total review counts of each app. The result will be assigned to a new variable `google_agg`. Here is the ([documentation on how to achieve it](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.agg.html)). Take a moment or two to read the documentation and google examples because it is pretty complex.

When you obtain `google_agg`, check its values to make sure it has an `App` column as its index as well as a `Positive` column and a `Total` column. Your output should look like:

![Positive Reviews Agg](positive-review-agg.png)

*Hint: Use `positive_function` you created earlier as part of the param passed to the `agg()` function in order to aggregate the number of positive reviews.*

#### Bonus:

As of Pandas v0.23.4, you may opt to supply an array or an object to `agg()`. If you use the array param, you'll need to rename the columns so that their names are `Positive` and `Total`. Using the object param will allow you to create the aggregated columns with the desirable names without renaming them. However, you will probably encounter a warning indicating supplying an object to `agg()` will become outdated. It's up to you which way you will use. Try both ways out. Any way is fine as long as it works.

In [ ]:
# Your code here:
google_agg = review_missing_removed.groupby('App').agg({'Positive': 'sum', 'Sentiment': 'count'})
google_agg = google_agg.rename(columns={'Sentiment': 'Total'})

Print the first 5 rows of `google_agg` to check it.

In [ ]:
# Your code here
google_agg

Positive  Total
App                                                                
10 Best Foods for You                                    162    194
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室                          31     40
11st                                                      23     39
1800 Contacts - Lens Store                                64     80
1LINE – One Line with One Touch                           27     38
...                                                      ...    ...
Hotels.com: Book Hotel Rooms & Find Vacation Deals        39     68
Hotspot Shield Free VPN Proxy & Wi-Fi Security            17     34
Hotstar                                                   14     32
Hotwire Hotel & Car Rental App                            16     33
Housing-Real Estate & Property                             8     21

[865 rows x 2 columns]

#### Add a derived column to `google_agg` that is the ratio of the `Positive` and the `Total` columns. Call this column `Positive Ratio`. 

Make sure to account for the case where the denominator is zero using the `np.where()` function.

In [ ]:
# Your code here:
google_agg['Positive Ratio'] = np.where(google_agg['Total'] != 0, google_agg['Positive']/google_agg['Total'], 0)
google_agg.head()

Positive  Total  Positive Ratio
App                                                              
10 Best Foods for You                  162    194        0.835052
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室        31     40        0.775000
11st                                    23     39        0.589744
1800 Contacts - Lens Store              64     80        0.800000
1LINE – One Line with One Touch         27     38        0.710526

#### Now drop the `Positive` and `Total` columns. Do this with `inplace=True`.

In [ ]:
# Your code here:
google_agg.drop(['Positive','Total'], axis=1, inplace=True)

Print the first 5 rows of `google_agg`. Your output should look like:

![Positive Reviews Agg](positive-review-ratio.png)

In [ ]:
# Your code here:
google_agg.head()

Positive Ratio
App                                             
10 Best Foods for You                   0.835052
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室        0.775000
11st                                    0.589744
1800 Contacts - Lens Store              0.800000
1LINE – One Line with One Touch         0.710526

# Challenge 3 - Join the Dataframes

In this part of the lab, we will join the two dataframes and obtain a dataframe that contains features we can use in our ML algorithm.

In the next cell, join the `google_missing_removed` dataframe with the `google_agg` dataframe on the `App` column. Assign this dataframe to the variable `google`.

In [ ]:
# Your code here:
google=review_missing_removed.merge(google_agg,left_on='App',right_on='App')

#### Let's look at the final result using the `head()` function. Your final product should look like:

![Final Product](google-final-head.png)

In [ ]:
# Your code here:
google.head()

App  ... Positive Ratio
0  10 Best Foods for You  ...       0.835052
1  10 Best Foods for You  ...       0.835052
2  10 Best Foods for You  ...       0.835052
3  10 Best Foods for You  ...       0.835052
4  10 Best Foods for You  ...       0.835052

[5 rows x 7 columns]

 # Challenge 4- Feature selection and modelling

Let's work with data about bank marketing.  You can find the dataset and description in the data folder
Please use RFE, RFECV and SelectFomModel  for selecting the features from your dataset.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from matplotlib import pylab
%matplotlib notebook
%matplotlib inline

Step 1. Check your data. Clean and encode them if necessary

In [ ]:
data=pd.read_excel('bank_marketing (1).xlsx')
data.head()

age        job  marital    education  default housing loan    contact  \
0   56  housemaid  married     basic.4y       no      no   no  telephone   
1   57   services  married  high.school  unknown      no   no  telephone   
2   37   services  married  high.school       no     yes   no  telephone   
3   40     admin.  married     basic.6y       no      no   no  telephone   
4   56   services  married  high.school       no      no  yes  telephone   

  month day_of_week  ...  campaign  pdays  previous     poutcome emp.var.rate  \
0   may         mon  ...         1    999         0  nonexistent   2022-01-01   
1   may         mon  ...         1    999         0  nonexistent   2022-01-01   
2   may         mon  ...         1    999         0  nonexistent   2022-01-01   
3   may         mon  ...         1    999         0  nonexistent   2022-01-01   
4   may         mon  ...         1    999         0  nonexistent   2022-01-01   

  cons.price.idx  cons.conf.idx  euribor3m  nr.employed   y  
0         93.994          -36.4      4.857       5191.0  no  
1         93.994          -36.4      4.857       5191.0  no  
2         93.994          -36.4      4.857       5191.0  no  
3         93.994          -36.4      4.857       5191.0  no  
4         93.994          -36.4      4.857       5191.0  no  

[5 rows x 21 columns]

In [ ]:
data.describe().T

count         mean         std       min       25%  \
age             41188.0    40.024060   10.421250    17.000    32.000   
duration        41188.0   258.285010  259.279249     0.000   102.000   
campaign        41188.0     2.567593    2.770014     1.000     1.000   
pdays           41188.0   962.475454  186.910907     0.000   999.000   
previous        41188.0     0.172963    0.494901     0.000     0.000   
cons.price.idx  41188.0    93.575664    0.578840    92.201    93.075   
cons.conf.idx   41188.0   -40.502600    4.628198   -50.800   -42.700   
euribor3m       41188.0     3.621291    1.734447     0.634     1.344   
nr.employed      7763.0  5191.000000    0.000000  5191.000  5191.000   

                     50%       75%       max  
age               38.000    47.000    98.000  
duration         180.000   319.000  4918.000  
campaign           2.000     3.000    56.000  
pdays            999.000   999.000   999.000  
previous           0.000     0.000     7.000  
cons.price.idx    93.749    93.994    94.767  
cons.conf.idx    -41.800   -36.400   -26.900  
euribor3m          4.857     4.961     5.045  
nr.employed     5191.000  5191.000  5191.000

In [ ]:
data.isnull().sum()

age                   0
job                   0
marital               0
education             0
default               0
housing               0
loan                  0
contact               0
month                 0
day_of_week           0
duration              0
campaign              0
pdays                 0
previous              0
poutcome              0
emp.var.rate      17191
cons.price.idx        0
cons.conf.idx         0
euribor3m             0
nr.employed       33425
y                     0
dtype: int64

In [ ]:
# Proportion of missing value:
data['emp.var.rate'].isna().sum()/data.shape[0]*100

41.7378848208216

In [ ]:
data['nr.employed'].isna().sum()/data.shape[0]*100

81.15227736233854

In [ ]:
# Large no of null value, so we can drop 
data.drop(columns=['emp.var.rate', 'nr.employed','cons.conf.idx','cons.price.idx'], inplace=True)

In [ ]:
data.isnull().sum()

age            0
job            0
marital        0
education      0
default        0
housing        0
loan           0
contact        0
month          0
day_of_week    0
duration       0
campaign       0
pdays          0
previous       0
poutcome       0
euribor3m      0
y              0
dtype: int64

In [ ]:
data.dtypes

age              int64
job             object
marital         object
education       object
default         object
housing         object
loan            object
contact         object
month           object
day_of_week     object
duration         int64
campaign         int64
pdays            int64
previous         int64
poutcome        object
euribor3m      float64
y               object
dtype: object

In [ ]:
for col in data.columns:
	print(data[col].unique())

[56 57 37 40 45 59 41 24 25 29 35 54 46 50 39 30 55 49 34 52 58 32 38 44
 42 60 53 47 51 48 33 31 43 36 28 27 26 22 23 20 21 61 19 18 70 66 76 67
 73 88 95 77 68 75 63 80 62 65 72 82 64 71 69 78 85 79 83 81 74 17 87 91
 86 98 94 84 92 89]
['housemaid' 'services' 'admin.' 'blue-collar' 'technician' 'retired'
 'management' 'unemployed' 'self-employed' 'unknown' 'entrepreneur'
 'student']
['married' 'single' 'divorced' 'unknown']
['basic.4y' 'high.school' 'basic.6y' 'basic.9y' 'professional.course'
 'unknown' 'university.degree' 'illiterate']
['no' 'unknown' 'yes']
['no' 'yes' 'unknown']
['no' 'yes' 'unknown']
['telephone' 'cellular']
['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'mar' 'apr' 'sep']
['mon' 'tue' 'wed' 'thu' 'fri']
[ 261  149  226 ... 1246 1556 1868]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 19 18 23 14 22 25 16 17 15 20 56
 39 35 42 28 26 27 32 21 24 29 31 30 41 37 40 33 34 43]
[999   6   4   3   5   1   0  10   7   8   9  11   2  12  13  14  15  16
  21  17  18  22  25  26  19

In [ ]:
data_for_feature = pd.get_dummies(data=data, drop_first=True)

In [ ]:
data_for_feature.dtypes

age                                int64
duration                           int64
campaign                           int64
pdays                              int64
previous                           int64
euribor3m                        float64
job_blue-collar                    uint8
job_entrepreneur                   uint8
job_housemaid                      uint8
job_management                     uint8
job_retired                        uint8
job_self-employed                  uint8
job_services                       uint8
job_student                        uint8
job_technician                     uint8
job_unemployed                     uint8
job_unknown                        uint8
marital_married                    uint8
marital_single                     uint8
marital_unknown                    uint8
education_basic.6y                 uint8
education_basic.9y                 uint8
education_high.school              uint8
education_illiterate               uint8
education_profes

In [ ]:
data_for_feature.shape

(41188, 50)

In [ ]:
data_for_feature.head()

age  duration  campaign  pdays  previous  euribor3m  job_blue-collar  \
0   56       261         1    999         0      4.857                0   
1   57       149         1    999         0      4.857                0   
2   37       226         1    999         0      4.857                0   
3   40       151         1    999         0      4.857                0   
4   56       307         1    999         0      4.857                0   

   job_entrepreneur  job_housemaid  job_management  ...  month_nov  month_oct  \
0                 0              1               0  ...          0          0   
1                 0              0               0  ...          0          0   
2                 0              0               0  ...          0          0   
3                 0              0               0  ...          0          0   
4                 0              0               0  ...          0          0   

   month_sep  day_of_week_mon  day_of_week_thu  day_of_week_tue  \
0          0                1                0                0   
1          0                1                0                0   
2          0                1                0                0   
3          0                1                0                0   
4          0                1                0                0   

   day_of_week_wed  poutcome_nonexistent  poutcome_success  y_yes  
0                0                     1                 0      0  
1                0                     1                 0      0  
2                0                     1                 0      0  
3                0                     1                 0      0  
4                0                     1                 0      0  

[5 rows x 50 columns]

Step 2. List your features

In [ ]:
X = data_for_feature.drop('y_yes', axis = 1)
y = data_for_feature['y_yes']

In [ ]:
X.shape

(41188, 49)

In [ ]:
X.columns

Index(['age', 'duration', 'campaign', 'pdays', 'previous', 'euribor3m',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_married', 'marital_single', 'marital_unknown',
       'education_basic.6y', 'education_basic.9y', 'education_high.school',
       'education_illiterate', 'education_professional.course',
       'education_university.degree', 'education_unknown', 'default_unknown',
       'default_yes', 'housing_unknown', 'housing_yes', 'loan_unknown',
       'loan_yes', 'contact_telephone', 'month_aug', 'month_dec', 'month_jul',
       'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct',
       'month_sep', 'day_of_week_mon', 'day_of_week_thu', 'day_of_week_tue',
       'day_of_week_wed', 'poutcome_nonexistent', 'poutcome_success'],
      dtype='object')

In [ ]:
y

0        0
1        0
2        0
3        0
4        0
        ..
41183    1
41184    0
41185    0
41186    1
41187    0
Name: y_yes, Length: 41188, dtype: uint8

In [ ]:
#standarization of the features
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x = sc.fit_transform(X)

Step 3. Apply the RandomForestClassifier and LogisticRegression model with default parameters to your data 
What is the accuracy for your models?

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2, stratify=y)
print (X_train.shape)
print (X_test.shape)
print (y_train.shape)
print (y_test.shape)

(28831, 49)
(12357, 49)
(28831,)
(12357,)


In [ ]:
from sklearn import svm
clf = LogisticRegression()
clf

LogisticRegression()

In [ ]:
clf.fit(X_train, y_train)

LogisticRegression()

In [ ]:
y_pred = clf.predict(X_test)
y_pred

array([1, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [ ]:
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred))

0.90928218823339


In [ ]:
# Imports
from sklearn.metrics import confusion_matrix, classification_report

# Confusion matrix
print(confusion_matrix(y_test, y_pred))

# New line
print('\n')

# Classification report
print(classification_report(y_test,y_pred))

[[10684   281]
 [  840   552]]


              precision    recall  f1-score   support

           0       0.93      0.97      0.95     10965
           1       0.66      0.40      0.50      1392

    accuracy                           0.91     12357
   macro avg       0.79      0.69      0.72     12357
weighted avg       0.90      0.91      0.90     12357



In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 100)
# Train the model on training data
rf.fit(X_train, y_train)

RandomForestRegressor()

In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)
# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred))

Mean Absolute Error: 0.11 degrees.
0.90928218823339


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Confusion matrix
print(confusion_matrix(y_test, y_pred))

# New line
print('\n')

# Classification report
print(classification_report(y_test,y_pred))

[[10684   281]
 [  840   552]]


              precision    recall  f1-score   support

           0       0.93      0.97      0.95     10965
           1       0.66      0.40      0.50      1392

    accuracy                           0.91     12357
   macro avg       0.79      0.69      0.72     12357
weighted avg       0.90      0.91      0.90     12357




Step 4. Select features using SelectFromModel method. Explain, how you define the optimal number of features


In [ ]:
from sklearn.feature_selection import SelectFromModel
SFM = SelectFromModel(estimator=RandomForestClassifier())
s=SFM.fit(x, y)
n_features = s.transform(x).shape[1]
n_features

6

In [ ]:
feature_idx = s.get_support()
feature_name = X.columns[feature_idx]
feature_name

Index(['age', 'duration', 'campaign', 'pdays', 'euribor3m',
       'poutcome_success'],
      dtype='object')

In [ ]:
SFM = SelectFromModel(estimator=LogisticRegression())
s_lr=SFM.fit(x, y)
n_features = s_lr.transform(x).shape[1]
n_features

13

In [ ]:
feature_idx = s_lr.get_support()
feature_name = X.columns[feature_idx]
feature_name

Index(['duration', 'campaign', 'pdays', 'euribor3m', 'job_blue-collar',
       'default_unknown', 'month_aug', 'month_jul', 'month_jun', 'month_mar',
       'month_may', 'poutcome_nonexistent', 'poutcome_success'],
      dtype='object')

In [ ]:
new_x=X[feature_name]
new_x.head()

duration  campaign  pdays  euribor3m  job_blue-collar  default_unknown  \
0       261         1    999      4.857                0                0   
1       149         1    999      4.857                0                1   
2       226         1    999      4.857                0                0   
3       151         1    999      4.857                0                0   
4       307         1    999      4.857                0                0   

   month_aug  month_jul  month_jun  month_mar  month_may  \
0          0          0          0          0          1   
1          0          0          0          0          1   
2          0          0          0          0          1   
3          0          0          0          0          1   
4          0          0          0          0          1   

   poutcome_nonexistent  poutcome_success  
0                     1                 0  
1                     1                 0  
2                     1                 0  
3                     1                 0  
4                     1                 0

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(new_x, y, test_size=0.2, random_state=5)


Step 5. Apply the RandomForestClassifier and LogisticRegression model with default parameters to your data (only selected features). What is the accuracy for your models?

In [ ]:
# Fitting LR
lr = LogisticRegression(random_state = 42, solver='lbfgs', multi_class='auto')
lr.fit(x_train, y_train)

y_pred = lr.predict(x_test)
probs_y=lr.predict_proba(x_test)
probs_y = np.round(probs_y, 2)

print('The accuracy of the Logistic Regression is:', lr.score(x_test, y_test)*100)

The accuracy of the Logistic Regression is: 90.32532168001943


C:\Users\manu2\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# Fitting RFC
rfc = RandomForestClassifier(n_estimators = 100)
rfc.fit(x_train, y_train)
y_pred = rfc.predict(x_test)
acc_rfc = rfc.score(x_test, y_test)

print('The accuracy of the Random Forest Classifier is', acc_rfc*100)

The accuracy of the Random Forest Classifier is 90.13109978150037



Step 6. Select features using RFE and RFECV methods. Explain, how you define the optimal number of features for each of the cases.

In [ ]:
#RFE
from sklearn.feature_selection import RFE
rfe_selector = RFE(estimator=LogisticRegression(),n_features_to_select = None, step = 1)
m=rfe_selector.fit(x, y)
X.columns[m.get_support()]
print("Num Features: %s" % (m.n_features_))
print("Selected Features: %s" % (m.support_))
print("Feature Ranking: %s" % (m.ranking_))

Num Features: 24
Selected Features: [False  True  True  True False  True  True False False False  True False
  True  True False False False False False False False False False False
 False  True False  True  True False False False False False  True  True
  True  True  True  True  True  True  True  True False False  True  True
  True]
Feature Ranking: [ 9  1  1  1  4  1  1  2 24 12  1  6  1  1 15 23 25 22  8 26 14 20 16 13
 10  1  5  1  1 18 19 21 11  7  1  1  1  1  1  1  1  1  1  1 17  3  1  1
  1]


In [ ]:
# RFECV
rfe_selector = RFE(estimator=RandomForestClassifier(),n_features_to_select = None, step = 1)
m=rfe_selector.fit(x, y)
X.columns[m.get_support()]
print("Num Features: %s" % (m.n_features_))
print("Selected Features: %s" % (m.support_))
print("Feature Ranking: %s" % (m.ranking_))

Num Features: 24
Selected Features: [ True  True  True  True  True  True  True False False False False False
 False False  True False False  True  True False False False  True False
 False  True False False False False  True False  True  True False False
 False  True  True  True False  True False  True  True  True  True False
  True]
Feature Ranking: [ 1  1  1  1  1  1  1 17 20  6 11 15  8 13  1 18 23  1  1 24 16  4  1 25
  2  1 12  5 26 19  1 22  1  1  7 21  9  1  1  1 10  1 14  1  1  1  1  3
  1]


In [ ]:
X.columns

Index(['age', 'duration', 'campaign', 'pdays', 'previous', 'euribor3m',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_married', 'marital_single', 'marital_unknown',
       'education_basic.6y', 'education_basic.9y', 'education_high.school',
       'education_illiterate', 'education_professional.course',
       'education_university.degree', 'education_unknown', 'default_unknown',
       'default_yes', 'housing_unknown', 'housing_yes', 'loan_unknown',
       'loan_yes', 'contact_telephone', 'month_aug', 'month_dec', 'month_jul',
       'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct',
       'month_sep', 'day_of_week_mon', 'day_of_week_thu', 'day_of_week_tue',
       'day_of_week_wed', 'poutcome_nonexistent', 'poutcome_success'],
      dtype='object')

In [ ]:
## RFECV_RANDOM_FOREST
from sklearn.feature_selection import RFECV


rfe = RFECV(
    estimator=RandomForestClassifier(),
    min_features_to_select=9,
    step=5,
    n_jobs=-1,
    verbose=1,
    scoring="r2",
    cv=5,
)

m= rfe.fit(x, y)
features_rfe = X.columns[rfe.support_]

Fitting estimator with 49 features.
Fitting estimator with 44 features.
Fitting estimator with 39 features.
Fitting estimator with 34 features.
Fitting estimator with 29 features.
Fitting estimator with 24 features.


In [ ]:
## RFECV_Logistics_Regression
from sklearn.feature_selection import RFECV


rfecv = RFECV(
    estimator=LogisticRegression(),
    min_features_to_select=9,
    step=5,
    n_jobs=-1,
    verbose=1,
    scoring="r2",
    cv=5,
)

m= rfecv.fit(x, y)
features_rfecv = X.columns[rfecv.support_]

Fitting estimator with 49 features.
Fitting estimator with 44 features.
Fitting estimator with 39 features.
Fitting estimator with 34 features.
Fitting estimator with 29 features.
Fitting estimator with 24 features.
Fitting estimator with 19 features.
Fitting estimator with 14 features.


In [ ]:
features_rfecv

Index(['duration', 'pdays', 'euribor3m', 'job_blue-collar', 'month_jul',
       'month_mar', 'month_may', 'poutcome_nonexistent', 'poutcome_success'],
      dtype='object')

In [ ]:
features_rfe

Index(['age', 'duration', 'campaign', 'pdays', 'previous', 'euribor3m',
       'job_technician', 'marital_married', 'education_high.school',
       'education_university.degree', 'housing_yes', 'loan_yes',
       'contact_telephone', 'month_mar', 'day_of_week_mon', 'day_of_week_thu',
       'day_of_week_tue', 'day_of_week_wed', 'poutcome_success'],
      dtype='object')

In [ ]:
### In this logistics regression works faster

Step 7. Apply the RandomForestClassifier and LogisticRegression model with default parameters to your data (you will have 4 models, taking into account two sets of features that you got). What is the accuracy for your models?


In [ ]:
### Check the model with logistics regression feature selection

In [ ]:
new_x=X[features_rfecv]
new_x.head()

duration  pdays  euribor3m  job_blue-collar  month_jul  month_mar  \
0       261    999      4.857                0          0          0   
1       149    999      4.857                0          0          0   
2       226    999      4.857                0          0          0   
3       151    999      4.857                0          0          0   
4       307    999      4.857                0          0          0   

   month_may  poutcome_nonexistent  poutcome_success  
0          1                     1                 0  
1          1                     1                 0  
2          1                     1                 0  
3          1                     1                 0  
4          1                     1                 0

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(new_x, y, test_size = 0.2, random_state=42)

In [ ]:
# Fitting LR
lr = LogisticRegression(random_state = 42, solver='lbfgs', multi_class='auto')
lr.fit(x_train, y_train)

y_pred = lr.predict(x_test)
probs_y=lr.predict_proba(x_test)
probs_y = np.round(probs_y, 2)

print('The accuracy of the Logistic Regression is:', lr.score(x_test, y_test))

The accuracy of the Logistic Regression is: 0.9085943190094683


C:\Users\manu2\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
rfc = RandomForestClassifier(n_estimators = 100)
rfc.fit(x_train, y_train)
y_pred = rfc.predict(x_test)
acc_rfc = rfc.score(x_test, y_test)

print('The accuracy of the Random Forest Classifier is', acc_rfc)

The accuracy of the Random Forest Classifier is 0.8986404467103666


In [ ]:
#### Second, check the model with random forest feature selection: 

In [ ]:
new_x=X[features_rfe]
new_x.head()

age  duration  campaign  pdays  previous  euribor3m  job_technician  \
0   56       261         1    999         0      4.857               0   
1   57       149         1    999         0      4.857               0   
2   37       226         1    999         0      4.857               0   
3   40       151         1    999         0      4.857               0   
4   56       307         1    999         0      4.857               0   

   marital_married  education_high.school  education_university.degree  \
0                1                      0                            0   
1                1                      1                            0   
2                1                      1                            0   
3                1                      0                            0   
4                1                      1                            0   

   housing_yes  loan_yes  contact_telephone  month_mar  day_of_week_mon  \
0            0         0                  1          0                1   
1            0         0                  1          0                1   
2            1         0                  1          0                1   
3            0         0                  1          0                1   
4            0         1                  1          0                1   

   day_of_week_thu  day_of_week_tue  day_of_week_wed  poutcome_success  
0                0                0                0                 0  
1                0                0                0                 0  
2                0                0                0                 0  
3                0                0                0                 0  
4                0                0                0                 0

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(new_x, y, test_size = 0.2, random_state=42)

In [ ]:
# Fitting LR
lr = LogisticRegression(random_state = 42, solver='lbfgs', multi_class='auto')
lr.fit(x_train, y_train)

y_pred = lr.predict(x_test)
probs_y=lr.predict_proba(x_test)
probs_y = np.round(probs_y, 2)

print('The accuracy of the Logistic Regression is:', lr.score(x_test, y_test))

The accuracy of the Logistic Regression is: 0.9104151493080845


C:\Users\manu2\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
rfc = RandomForestClassifier(n_estimators = 100)
rfc.fit(x_train, y_train)
y_pred = rfc.predict(x_test)
acc_rfc = rfc.score(x_test, y_test)

print('The accuracy of the Random Forest Classifier is', acc_rfc)

The accuracy of the Random Forest Classifier is 0.9150279193979121











Step 8. Please make the conclusion about the utility of the feature selection

In [ ]:
Summary_models=pd.DataFrame({'Name of the models':['Logistics_Regression', 'Random_Forest'], 
                             'Default_Features':['90.9','90.9'],'Selected_Features':['90.3','90.1'],
                             'RFE':['91.0','91.5'],'RFECV':['90.8','89.8']})
Summary_models

Name of the models Default_Features Selected_Features   RFE RFECV
0  Logistics_Regression             90.9              90.3  91.0  90.8
1         Random_Forest             90.9              90.1  91.5  89.8